# Retrieval augmented generation

In [1]:
import os 
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
os.environ['OPENAI_API_KEY']=os.getenv('OPENAI_API_KEY')

In [3]:
from llama_index import VectorStoreIndex, SimpleDirectoryReader

documents = SimpleDirectoryReader("data").load_data()

In [4]:
index = VectorStoreIndex.from_documents(documents, show_progress=True)

/home/renan/src-mydev/llm/llamaindex/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating embeddings: 100%|██████████| 26/26 [00:02<00:00,  8.83it/s]


In [5]:
index

In [6]:
query_engine = index.as_query_engine()

In [7]:
query_engine

In [8]:
response = query_engine.query("What is open-vocabulary object dection?")

In [9]:
print(response)

Open-vocabulary object detection refers to the task of detecting objects beyond the predefined categories. Traditional object detection methods are limited to detecting objects from a fixed vocabulary, but open-vocabulary object detection aims to detect and recognize novel objects that are not part of the predefined categories. This approach allows for greater flexibility and generalization in detecting objects in various scenarios and domains.


In [10]:
from llama_index.response.pprint_utils import pprint_response

pprint_response(response)

Final Response: Open-vocabulary object detection refers to the task of
detecting objects beyond the predefined categories. Traditional object
detection methods are limited to detecting objects from a fixed
vocabulary, but open-vocabulary object detection aims to detect and
recognize novel objects that are not part of the predefined
categories. This approach allows for greater flexibility and
generalization in detecting objects in various scenarios and domains.


In [11]:
pprint_response(response, show_source=True)

Final Response: Open-vocabulary object detection refers to the task of
detecting objects beyond the predefined categories. Traditional object
detection methods are limited to detecting objects from a fixed
vocabulary, but open-vocabulary object detection aims to detect and
recognize novel objects that are not part of the predefined
categories. This approach allows for greater flexibility and
generalization in detecting objects in various scenarios and domains.
______________________________________________________________________
Source Node 1/2
Node ID: 7a487229-9c2d-4628-ab32-bcb98957b8d4
Similarity: 0.8554250535022178
Text: During the past decades, the methods for traditional object de-
tection can be simply categorized into three groups, i.e., region-
based methods, pixel-based methods, and query- based methods. The
region-based methods [10, 11, 15, 26, 41], such as Faster R-CNN [41],
adopt a two-stage frame- work for proposal generation [41] and RoI-
wise (Region- ...
____________

In [12]:
from llama_index.retrievers import VectorIndexRetriever
from llama_index.query_engine import RetrieverQueryEngine
from llama_index.indices.postprocessor import SimilarityPostprocessor

retriever = VectorIndexRetriever(index=index, similarity_top_k=4)

query_engine = RetrieverQueryEngine(retriever=retriever)


In [13]:
response = query_engine.query("What is open-vocabulary object dection?")

In [14]:
pprint_response(response, show_source=True)

Final Response: Open-vocabulary object detection refers to the task of
detecting objects beyond the predefined categories. Traditional object
detection methods can only detect objects within a fixed vocabulary of
predefined categories. However, open-vocabulary object detection aims
to detect and recognize novel objects that are not part of the
predefined categories. This allows for greater flexibility and
generalization in detecting objects in various scenarios and domains.
______________________________________________________________________
Source Node 1/4
Node ID: 7a487229-9c2d-4628-ab32-bcb98957b8d4
Similarity: 0.8554250535022178
Text: During the past decades, the methods for traditional object de-
tection can be simply categorized into three groups, i.e., region-
based methods, pixel-based methods, and query- based methods. The
region-based methods [10, 11, 15, 26, 41], such as Faster R-CNN [41],
adopt a two-stage frame- work for proposal generation [41] and RoI-
wise (Region- ..

In [15]:
retriever = VectorIndexRetriever(index=index, similarity_top_k=4)
postprocessor = SimilarityPostprocessor(similarity_cutoff=0.85)
query_engine = RetrieverQueryEngine(retriever=retriever, node_postprocessors=[postprocessor])


In [16]:
response = query_engine.query("What is open-vocabulary object dection?")
pprint_response(response, show_source=True)

Final Response: Open-vocabulary object detection refers to the task of
detecting objects beyond the predefined categories. Traditional object
detection methods are limited to detecting objects from a fixed
vocabulary, but open-vocabulary object detection aims to detect and
recognize novel objects that are not part of the predefined
categories. This approach allows for greater flexibility and
generalization in detecting objects in various scenarios and domains.
______________________________________________________________________
Source Node 1/2
Node ID: 7a487229-9c2d-4628-ab32-bcb98957b8d4
Similarity: 0.8554250535022178
Text: During the past decades, the methods for traditional object de-
tection can be simply categorized into three groups, i.e., region-
based methods, pixel-based methods, and query- based methods. The
region-based methods [10, 11, 15, 26, 41], such as Faster R-CNN [41],
adopt a two-stage frame- work for proposal generation [41] and RoI-
wise (Region- ...
____________

# Second Part - Persist index

In [17]:
import os.path
from llama_index import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    StorageContext,
    load_index_from_storage
)

PERSISTENCE_DIR = "./storage"

# If the index has not been persisted yet, create it and persist it
if not os.path.exists(PERSISTENCE_DIR):
    documents = SimpleDirectoryReader("data").load_data()
    index = VectorStoreIndex.from_documents(documents)

    index.storage_context.persist(PERSISTENCE_DIR)
# If the index has been persisted, load it from storage    
else:
    storage_context = StorageContext.from_defaults(persist_dir=PERSISTENCE_DIR)
    index = load_index_from_storage(storage_context)

# Create a query engine from the index
query_engine = index.as_query_engine()
response = query_engine.query("What is open-vocabulary object dection?")

pprint_response(response, show_source=True)



Final Response: Open-vocabulary object detection refers to the task of
detecting objects beyond the predefined categories. It aims to detect
and recognize novel objects that are not part of the limited dataset
and vocabulary used for training. This approach allows for greater
flexibility and generalization in detecting objects in open scenarios
and different domains.
______________________________________________________________________
Source Node 1/2
Node ID: 763b56dc-c00a-4444-9c0d-3060c0d60ce0
Similarity: 0.8554250535022178
Text: During the past decades, the methods for traditional object de-
tection can be simply categorized into three groups, i.e., region-
based methods, pixel-based methods, and query- based methods. The
region-based methods [10, 11, 15, 26, 41], such as Faster R-CNN [41],
adopt a two-stage frame- work for proposal generation [41] and RoI-
wise (Region- ...
______________________________________________________________________
Source Node 2/2
Node ID: dbb3abd3-53